Install libraries

RAG Stage: Environment Setup
We install all required libraries that will give us the ability to load documents, split them, convert them into embeddings (vectors), store them in a vector database, and run an LLM locally for Q/A.

In [ ]:
!pip install langchain langchain-community chromadb sentence-transformers transformers accelerate


Load website

RAG Stage: Document Loading (Knowledge Source Collection)
We load a public webpage from Python documentation. The loader extracts visible text from the website and stores it in a list of Document objects (docs), which will later be chunked and embedded.

In [ ]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(web_paths=["https://docs.python.org/3/tutorial/introduction.html#lists"])
docs = loader.load()

print("Pages loaded:", len(docs))
print("\nPreview:\n", docs[0].page_content[:300])


Pages loaded: 1

Preview:
 
















3. An Informal Introduction to Python — Python 3.14.2 documentation




















































    Theme
    
Auto
Light
Dark



Table of Contents

3. An Informal Introduction to Python
3.1. Using Python as a Calculator
3.1.1. Numbers
3.1.2. Text
3.1.3. List


Split into chunks

RAG Stage: Chunking (Text Preprocessing for Retrieval)
Large text is harder to embed and search. So we split the webpage text into smaller overlapping chunks (500 characters each, 50 characters overlap) so that each chunk contains meaningful, searchable information without losing continuity.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

splits = splitter.split_documents(docs)

print("Total chunks created:", len(splits))
print("\nExample chunk:\n", splits[0].page_content[:200])

Total chunks created: 52

Example chunk:
 3. An Informal Introduction to Python — Python 3.14.2 documentation




















































    Theme
    
Auto
Light
Dark



Table of Contents

3. An Informal Introduction


Load embedding model

RAG Stage: Embedding Creation (Convert text → meaning vectors)
We load a lightweight open embedding model from Sentence Transformers. This model will convert each chunk of text into a numerical vector that captures its semantic meaning. These vectors are used for similarity search in retrieval.

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

print("Embedding model loaded!")


Embedding model loaded!


Store vectors in ChromaDB + Initialize Retriever

RAG Stage: Indexing & Retrieval Setup
We convert each text chunk into an embedding vector and store it in ChromaDB. We then initialize a retriever, which is a meaning-based search tool that will later fetch the most relevant chunks for any user question.

In [ ]:
from langchain_community.vectorstores import Chroma

vector_db = Chroma.from_documents(
    documents=splits,
    embedding=embedding_model
)

retriever = vector_db.as_retriever()

print("Vector database created!")
print("Total vectors stored:", vector_db._collection.count())


Vector database created!
Total vectors stored: 72


Retrieve context for a question

RAG Stage: Retrieval (Search by meaning and fetch relevant context)
We input a question. The retriever converts it into an embedding internally and finds the most semantically similar vectors from the DB. It returns the top matching chunks as Document objects, which we print to verify grounding context.

In [ ]:
question = "What are Python lists?"

retrieved_docs = retriever.invoke(question)

print("Retrieved knowledge chunks:\n")
for i, doc in enumerate(retrieved_docs):
    print(i+1, "->", doc.page_content[:150], "\n")

Retrieved knowledge chunks:

1 -> 3.1.3. Lists¶
Python knows a number of compound data types, used to group together other
values.  The most versatile is the list, which can be written 

2 -> language’s flavor and style. After reading it, you will be able to read and
write Python modules and programs, and you will be ready to learn more abo 

3 -> 4.9.4. Arbitrary Argument Lists
4.9.5. Unpacking Argument Lists
4.9.6. Lambda Expressions
4.9.7. Documentation Strings
4.9.8. Function Annotations


4 

4 -> This tutorial introduces the reader informally to the basic concepts and
features of the Python language and system. Be aware that it expects you to
h 



Load the LLM

RAG Stage: LLM Setup (Load free open Q/A-capable model)
We now load a small open-source LLM trained for Q/A and prompt following. flan-t5-base is fully open (not gated), fast, and better than GPT-2 for instruction-based answers. It will generate answers from our augmented prompt.

In [ ]:
from transformers import pipeline

llm = pipeline(
    "text2text-generation",
    model="google/flan-t5-base",
    max_new_tokens=150
)

print("LLM loaded!")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


LLM loaded!


Augment prompt and generate answer

RAG Stage: Augmented Generation (Insert retrieved context into prompt and answer)
We combine retrieved chunks into one text block, build a prompt containing only that context and the question, and pass it to the LLM. The LLM reads the real website knowledge and generates a grounded answer from it.

In [ ]:
context = "\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
Answer the question using only the context below.

Context:
{context}

Question: {question}

Answer:
"""

print("Prompt sent to LLM:\n", prompt)

response = llm(prompt)
print("\nFinal Answer:\n", response[0]["generated_text"])


Prompt sent to LLM:
 
Answer the question using only the context below.

Context:
3.1.3. Lists¶
Python knows a number of compound data types, used to group together other
values.  The most versatile is the list, which can be written as a list of
comma-separated values (items) between square brackets.  Lists might contain
items of different types, but usually the items all have the same type.
>>> squares = [1, 4, 9, 16, 25]
>>> squares
[1, 4, 9, 16, 25]
language’s flavor and style. After reading it, you will be able to read and
write Python modules and programs, and you will be ready to learn more about the
various Python library modules described in The Python Standard Library.
The Glossary is also worth going through.
4.9.4. Arbitrary Argument Lists
4.9.5. Unpacking Argument Lists
4.9.6. Lambda Expressions
4.9.7. Documentation Strings
4.9.8. Function Annotations


4.10. Intermezzo: Coding Style


5. Data Structures
5.1. More on Lists
5.1.1. Using Lists as Stacks
5.1.2. Using Lists as 

In [ ]:
# -------- RAG Stage: Retrieval --------
question = "Explain Python lists in simple words"

retrieved_docs = retriever.invoke(question)

print("Retrieved Chunks:\n")
for i, doc in enumerate(retrieved_docs):
    print(i+1, "->", doc.page_content[:150], "\n")

# -------- RAG Stage: Augmentation --------
context = "\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
Answer the question using only the context below.

Context:
{context}

Question: {question}

Answer:
"""

print("Prompt sent to LLM:\n", prompt)

# -------- RAG Stage: Generation --------
response = llm(prompt)

print("\nFinal Answer:\n", response[0]["generated_text"])


Retrieved Chunks:

1 -> 3.1.3. Lists¶
Python knows a number of compound data types, used to group together other
values.  The most versatile is the list, which can be written 

2 -> This tutorial introduces the reader informally to the basic concepts and
features of the Python language and system. Be aware that it expects you to
h 

3 -> language’s flavor and style. After reading it, you will be able to read and
write Python modules and programs, and you will be ready to learn more abo 

4 -> Simple assignment in Python never copies data. When you assign a list
to a variable, the variable refers to the existing list.
Any changes you make to 

Prompt sent to LLM:
 
Answer the question using only the context below.

Context:
3.1.3. Lists¶
Python knows a number of compound data types, used to group together other
values.  The most versatile is the list, which can be written as a list of
comma-separated values (items) between square brackets.  Lists might contain
items of different types, 

In [ ]:
# -------- RAG Stage: Retrieval --------
question = "How do Python loops work?"

retrieved_docs = retriever.invoke(question)

print("Retrieved Chunks:\n")
for i, doc in enumerate(retrieved_docs):
    print(i+1, "->", doc.page_content[:150], "\n")

# -------- RAG Stage: Augmentation --------
context = "\n".join([doc.page_content for doc in retrieved_docs])

prompt = f"""
Answer the question using only the context below.

Context:
{context}

Question: {question}

Answer:
"""

print("Prompt sent to LLM:\n", prompt)

# -------- RAG Stage: Generation --------
response = llm(prompt)

print("\nFinal Answer:\n", response[0]["generated_text"])


Retrieved Chunks:

1 -> The first line contains a multiple assignment: the variables a and b
simultaneously get the new values 0 and 1.  On the last line this is used again,
 

2 -> |







The Python Tutorial¶

Tip
This tutorial is designed for
programmers that are new to the Python language,
not beginners who are new to program 

3 -> In the Python shell, the string definition and output string can look
different.  The print() function produces a more readable output, by
omitting th 

4 -> This tutorial introduces the reader informally to the basic concepts and
features of the Python language and system. Be aware that it expects you to
h 

Prompt sent to LLM:
 
Answer the question using only the context below.

Context:
The first line contains a multiple assignment: the variables a and b
simultaneously get the new values 0 and 1.  On the last line this is used again,
demonstrating that the expressions on the right-hand side are all evaluated
first before any of the assignments take 